In [ ]:
import pandas as pd
import pickle, time, math, random
import tqdm.notebook as tqdm
import numpy as np
import matplotlib.pyplot as plt
import copy
from Trading_functions import * # where required functions defined

# data load - cannot upload on github, so I made a example df. 
# with open('./close_df.pkl', 'rb') as f:
#     close_df = pickle.load(f)
# with open('./volume_df.pkl', 'rb') as f:
#     volume_df = pickle.load(f)
# with open('./acml_df.pkl', 'rb') as f:
#     acml_df = pickle.load(f)
# with open('./low_df.pkl', 'rb') as f:
#     low_df = pickle.load(f)
# with open('./high_df.pkl', 'rb') as f:
#     high_df = pickle.load(f)
# with open('./open_df.pkl', 'rb') as f:
#     open_df = pickle.load(f)
# with open('./return_df.pkl', 'rb') as f:
#     return_df = pickle.load(f)

with open('df_index.pkl', 'rb') as f:
    df_index = pickle.load(f)
with open('df_columns.pkl', 'rb') as f:
    df_columns = pickle.load(f)
    
example_df = pd.DataFrame([], index = df_index, columns = df_columns) # all None data. 

# id_set
id_set = ['stck_clpr', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'acml_tr_pbmn']

# preprocess return data
return_df =  ((return_df+1).applymap(winsorize) * (return_df-return_df+1)) # winsorize to crop extreme data
return_df = ((acml_df==0)* (0.01)+ (acml_df>0)*return_df) # if not traded, give -99%
return_df = return_df.replace([np.inf, -np.inf], np.nan) # inf to nan
return_df[(close_df.notna().shift(1).fillna(False) * close_df.isna())] = 0.01 # if delisted, give -99%

# mask on
mask = (acml_df ==0).replace(True, np.nan).replace(False, 1)
close_df *= mask
volume_df *= mask
open_df *= mask
high_df *= mask
low_df *= mask
acml_df *= mask

# additional mask for enough volume
new_mask = volume_df.rolling(5, min_periods =1).min() > 10**7 * 5

In [ ]:
def replace_one(original, delete, change):
    rand_num = len(original.split(delete))-1
    rand_index = int(random.random()*rand_num)
    split = original.split(delete)
    split[rand_index] += (change + split[rand_index+1])
    del split[rand_index+1]
    return delete.join(split)

In [ ]:
class GA_Generator(GraphGenerator):
    def __init__(self, id_set, dic_operator):
        super().__init__(id_set, dic_operator)
    
    def random_expression(self, depth, max_depth):
        rand_value = random.random()
        if depth ==0:
            if random.random() < 1/3:
                depth +=1
                return 'BIGGER DOUBLE' + '(' + self.random_expression(depth, max_depth) + ',' + self.random_expression(depth, max_depth) + ')'
            else:
                depth +=1
                return np.random.choice(['BIGGER 0', 'BIGGER 1'], p=[0.5, 0.5]) + '(' + self.random_expression(depth, max_depth) + ')'
        if rand_value < depth/max_depth:
            rand_ind = int(random.random()*len(self.id_set))
            return 'GET' + '('+ self.id_set[rand_ind]+ ')'
        elif rand_value < 1 - depth/(max_depth):
            rand_ind = int(random.random()*len(self.p_modification))
            depth +=1
            return np.random.choice(list(self.dic_modification.keys()), p=self.p_modification) + '(' + self.random_expression(depth, max_depth) + ')'
        else:
            rand_ind = int(random.random()*len(self.p_combination))
            depth +=1
            return np.random.choice(list(self.dic_combination.keys()), p=self.p_combination) + '(' + self.random_expression(depth, max_depth) + ',' + self.random_expression(depth, max_depth) + ')'
        
    def calculate_depth(self, expression):
        operation = expression.split('(')[0]
        if operation in self.dic_imputer.keys():
            return 1
        elif operation in self.dic_modification.keys():
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            return self.calculate_depth(expression[bracket_1+1:len(expression)-bracket_2-1]) + 1
        elif operation in self.dic_combination.keys():
            counter = 0
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            inner = expression[bracket_1+1:len(expression)-bracket_2-1]
            for i in range(len(inner)):
                if (counter == 0) and inner[i] == ',':
                    first = inner[:i]
                    second = inner[i+1:]
                elif inner[i] == '(':
                    counter +=1
                elif inner[i] == ')':
                    counter -=1
            return max(self.calculate_depth(first), self.calculate_depth(second)) + 1
        elif operation == 'BIGGER DOUBLE':
            counter = 0
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            inner = expression[bracket_1+1:len(expression)-bracket_2-1]
            for i in range(len(inner)):
                if (counter == 0) and inner[i] == ',':
                    first = inner[:i]
                    second = inner[i+1:]
                elif inner[i] == '(':
                    counter +=1
                elif inner[i] == ')':
                    counter -=1
            return max(self.calculate_depth(first), self.calculate_depth(second)) + 1
        else:
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            return self.calculate_depth(expression[bracket_1+1:len(expression)-bracket_2-1]) + 1
        
    def peel_expression(self, expression):
        operation = expression.split('(')[0]
        if operation in self.dic_imputer.keys():
            return expression
        elif operation in self.dic_modification.keys():
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            return expression[bracket_1+1:len(expression)-bracket_2-1]
        elif operation in self.dic_combination.keys():
            counter = 0
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            inner = expression[bracket_1+1:len(expression)-bracket_2-1]
            for i in range(len(inner)):
                if (counter == 0) and inner[i] == ',':
                    first = inner[:i]
                    second = inner[i+1:]
                elif inner[i] == '(':
                    counter +=1
                elif inner[i] == ')':
                    counter -=1
            if random.random() < 0.5:
                return first
            else:
                return second
        elif operation == 'BIGGER DOUBLE':
            counter = 0
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            inner = expression[bracket_1+1:len(expression)-bracket_2-1]
            for i in range(len(inner)):
                if (counter == 0) and inner[i] == ',':
                    first = inner[:i]
                    second = inner[i+1:]
                elif inner[i] == '(':
                    counter +=1
                elif inner[i] == ')':
                    counter -=1
            if random.random() < 0.5:
                return first
            else:
                return second
        else:
            bracket_1 = expression.find('(')
            bracket_2 = expression[::-1].find(')')
            return expression[bracket_1+1:len(expression)-bracket_2-1]
        
    def peel_iteration(self, expression, iteration):
        for i in range(iteration):
            expression = self.peel_expression(expression)
        return expression
    
    def mix(self, expression1, expression2):
        depth1 = self.calculate_depth(expression1)
        depth2 = self.calculate_depth(expression2)
        max_depth = min(depth1, depth2) - 2 # minimum 1 
        change_depth = int(random.random()*max_depth) + 1
        injected = expression1
        ed_depth = depth1
        injector = expression2
        or_depth = depth2
        if random.random()< 0.5:
            injected, injector = injector, injected
            ed_depth, or_depth = or_depth, ed_depth
        delete_expression = self.peel_iteration(injected, ed_depth - change_depth)
        injected_expression = self.peel_iteration(injector, or_depth - change_depth)
        return replace_one(injected, delete_expression, injected_expression)
    
    def mutate(self, expression):
        depth = self.calculate_depth(expression) - 2
        change_depth = int(random.random()*depth) + 1
        delete_expression = self.peel_iteration(expression, change_depth)
        mutation = self.random_expression(1, depth+2 - change_depth)
        return replace_one(expression, delete_expression, mutation)

In [ ]:
class GA_child(object):
    def __init__(self):
        self.filters = []
        self.datas = []
        self.data = None
        
    def add(self, graph):
        self.filters.append(graph.expression)
        self.datas.append(graph.data)
        if type(self.data) == type(None):
            self.data = graph.data
        else:
            self.data *= graph.data
        self.data *= new_mask
        
    def delete(self):
        index = int(random.random() * len(self.filters))
        del self.filters[index]
        del self.datas[index]
        
    def get_one(self):
        index = int(random.random() * len(self.filters))
        return self.filters[index], index
    
    def update(self):
        self.data = self.datas[0]
        if len(self.datas)>1:
            for data in self.datas[1:]:
                self.data *= data
        self.data *= new_mask
    
    def get_case_data(self, engine, full, training):
        datas = []
        for expression in self.filters:
            datas.append(engine.expression_to_graph(expression, full, training).data)
        data = datas[0]
        if len(datas)>1:
            for data_2 in datas[1:]:
                data *= data_2
        if training=='training':
            case_mask = new_mask.loc['20060101':'20170101']
        elif training == 'validation':
            case_mask = new_mask.loc['20170101':'20210101']
        else:
            case_mask = new_mask.loc['20210101':]
        return data * case_mask
    
    def addition(self, child):
        if len(self.filters) + len(child.filters) <= 3:
            return self.filters + child.filters, self.datas + child.datas
        else:
            new_filters = []
            new_datas = []
            if len(self.filters) > len(child.filters):
                index = random.sample(list(range(len(self.filters))), 2)
                for i in index:
                    new_filters.append(self.filters[i])
                    new_datas.append(self.datas[i])
                    
                index = random.sample(list(range(len(child.filters))), 1)
                new_filters.append(child.filters[index[0]])
                new_datas.append(child.datas[index[0]])
                return new_filters, new_datas
            else:
                index = random.sample(list(range(len(self.filters))), 1)
                new_filters.append(self.filters[index[0]])
                new_datas.append(self.datas[index[0]])
                
                index = random.sample(list(range(len(child.filters))), 2)
                for i in index:
                    new_filters.append(child.filters[i])
                    new_datas.append(child.datas[i])
                return new_filters, new_datas
            
    def restore(self, filters):
        for exp in filters:
            self.add(engine.expression_to_graph(exp, True, 'training'))

def health_check(graph):
    graph.data *= new_mask
    if ((graph.data.sum(axis=1)/close_df.notna().sum(axis=1)).mean(0) > 0.4) or ((graph.data.sum(axis=1)/close_df.notna().sum(axis=1)).mean(0) < 0.1):
        return False
    else:
        return True

def cagr(cash_hist):
    return (cash_hist[-1]/cash_hist[0])**(252/len(cash_hist))-1

def stdv(returns):
    return np.std(returns)*math.sqrt(252)

def sharpe(returns):
    return cagr(returns.cumprod())/stdv(returns)

def transaction_cost(weight):
    cost = (((weight - weight.shift(-1)) > 0).astype(float) * (weight-weight.shift(-1))).sum(1) *0.0031
    return (1-cost).shift(1)

# GA run

children num : Pool size (Trade-off between performance and time)  
depth : expression tree size (Representation power)  

In [ ]:
children = []
engine = GA_Generator(id_set, dic_operator)

start_time = time.time()
# Make first Generation born
child_num = 30
child_num2 = child_num
tree_depth = 3
high_val = 0
count = 0

print('generation 0')
while child_num > 0:
    graph = engine.expression_to_graph(engine.random_expression(0,tree_depth), True, 'training')
    if health_check(graph):
        child = GA_child()
        child.add(graph)
        children.append(child)
        child_num -=1
    else:
        pass
child_num = child_num2

print('breeding completed')


print('Evolution starts')
# Start Evolution
for gen in range(100):
    miss_trial = 0

    # calculate
    cagr_list = []
    for child in children:
        weight = ((1/child.data.sum(axis=1)).replace([np.inf, -np.inf], np.nan).tolist()* child.data.T).T.shift(1)
        weight = weight.applymap(lambda x: 0.1 if x > 0.1 else x) # Maximum weight 0.1
        cost_discount = transaction_cost(weight)
        for_std = ((weight * (return_df.loc['20060101':'20170101'])).sum(axis=1)*cost_discount) + (1 - weight.sum(1))
        cagr_list.append(sharpe(for_std.iloc[1:]))


    # find Dominants
    dominant_num = int(child_num * 0.2)
    dom_index = np.argsort(cagr_list)[-dominant_num:]
    dominants = []
    for index in dom_index:
        dominants.append(children[index])
    print(gen, 'th generation dominant mean score:', np.mean(np.array(cagr_list)[dom_index]))

    # get Validation performace
    cagr_list_val = []
    for dominant in dominants:
        temp_data = dominant.get_case_data(engine, True, 'validation')
        weight = ((1/temp_data.sum(axis=1)).replace([np.inf, -np.inf], np.nan).tolist()* temp_data.T).T.shift(1)
        weight = weight.applymap(lambda x: 0.1 if x > 0.1 else x)
        cost_discount = transaction_cost(weight)
        for_std = ((weight * (return_df.loc['20170101':'20210101'])).sum(axis=1)*cost_discount) + (1-weight.sum(1))
        cagr_list_val.append(sharpe(for_std.iloc[1:]))
    print(gen, 'th generation dominant mean validation score:', np.mean(cagr_list_val))


    if np.mean(cagr_list_val)> high_val:
        high_val = np.mean(cagr_list_val)
    
    # dump
    with open('high_val_depth_'+str(count)+'.pkl', 'wb') as f:
        pickle.dump([dominant.filters for dominant in dominants], f)

    # make mix
    mix_num = int(child_num * 0.07)
    mix_list = []
    trial = 0
    while (mix_num > 0) and trial < 100:
        ind1, ind2 = int(random.random()*dominant_num), int(random.random()*dominant_num)
        exp1, child_ind1 = dominants[ind1].get_one()
        exp2, child_ind2 = dominants[ind2].get_one()
        mix_gen = engine.mix(exp1, exp2)
        graph = engine.expression_to_graph(mix_gen, True, 'training')
        new_gen = copy.deepcopy(dominants[ind1])
        new_gen.filters[child_ind1] = mix_gen
        new_gen.datas[child_ind1] = graph.data
        new_gen.update()
        if health_check(new_gen):
            mix_list.append(new_gen)
            mix_num -=1
        else:
            trial += 1
    if trial == 100:
        miss_trial += mix_num

    mutate_num = int(child_num * 0.1)
    mutate_list = []
    trial = 0
    while (mutate_num > 0) and trial < 100:
        ind = int(random.random()*dominant_num)
        exp, child_ind = dominants[ind].get_one()
        mutate_gen = engine.mutate(exp)
        graph = engine.expression_to_graph(mutate_gen, True, 'training')
        new_gen = copy.deepcopy(dominants[ind])
        new_gen.filters[child_ind] = mutate_gen
        new_gen.datas[child_ind] = graph.data
        new_gen.update()
        if health_check(new_gen):
            mutate_list.append(new_gen)
            mutate_num -=1
        else:
            trial +=1
    if trial == 100:
        miss_trial += mutate_num

    # make addition
    addition_num = int(child_num * 0.07)
    addition_list = []
    trial = 0
    while (addition_num > 0) and (trial < 100):
        ind1, ind2 = int(random.random()*dominant_num), int(random.random()*dominant_num)
        new_filters, new_datas = dominants[ind1].addition(dominants[ind2])
        new_gen = GA_child()
        new_gen.filters = new_filters
        new_gen.datas = new_datas
        new_gen.update()
        if health_check(new_gen):
            addition_list.append(new_gen)
            addition_num -=1
        else:
            trial+=1
    if trial == 100:
        miss_trial += addition_num

    # make new candidates
    new_num = int(child_num * 0.58) + miss_trial
    new_list = []
    while new_num > 0:
        graph = engine.expression_to_graph(engine.random_expression(0,tree_depth), True, 'training')
        if health_check(graph):
            new_child = GA_child()
            new_child.add(graph)
            new_list.append(new_child)
            new_num -=1
        else:
            pass
    children = dominants + mix_list + mutate_list + addition_list + new_list
    resets = []
    for i in range(len(children)):
        test_child = GA_child()
        test_child.restore(children[i].filters)
        resets.append(test_child)
    children = resets
    count+=1